# Playground for Riot Watcher

This is a testing notebook where I explore and test out features of `riotwatcher`. 

In [1]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import zilean

import json

api_key = zilean.read_api_key()
watcher = LolWatcher(api_key=api_key)
my_region = 'na1'
me = watcher.summoner.by_name(my_region, 'yENatividad')

## Match Timeline Summary

In [2]:
my_matches = watcher.match.matchlist_by_puuid(my_region, me['puuid'])
latest_matchid = my_matches[2]
latest_match = watcher.match.by_id(my_region, latest_matchid)
latest_timeline = watcher.match.timeline_by_match(my_region, latest_matchid)

In [7]:
summary = zilean.process_timeframe(latest_timeline, 10, latest_matchid)
pd.DataFrame([summary])

,level_0,timeEnemySpentControlled_0,totalGold_0,xp_0,magicDamageDone_0,magicDamageDoneToChampions_0,magicDamageTaken_0,physicalDamageDone_0,physicalDamageDoneToChampions_0,physicalDamageTaken_0,...,magicDamageTaken_4,physicalDamageDone_4,physicalDamageDoneToChampions_4,physicalDamageTaken_4,trueDamageDone_4,trueDamageDoneToChampions_4,trueDamageTaken_4,creepScore_4,matchId,win
0,-1,-26475,-987,-1003,-1769,-1415,1415,-6118,-932,3,...,-442,1222,59,859,-660,14,-98,7,NA1_4307172207,False
